In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
from src.etl import export_dict_to_csv

2025-10-15 01:26:11.206 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-10-15 01:26:11.206 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-10-15 01:26:11.206 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [2]:
df = pd.read_csv('data/Data raw.csv', delimiter=';')


In [3]:
lista_columnas = [" FECHA ", " INGRESOS_MATRICULAS ", " INGRESOS_RENOVACIONES ", " INGRESOS_ESAL ", " MATRICULA_COMERCIANTE ", " RENOVADOS_ESAL ", " RENOVADOS_COMERCIANTE ", " INGRESOS_RUP ", " INGRESOS_DERECHOS_DE_I ", " INGRESOS_RUNEOL ", " INGRESOS_FORMU_DE_REGISTRO_M ", " INGRESOS_SERVICIOS_ESPECIALES "]

In [4]:
df = df[lista_columnas]

In [5]:
# Renombrar columnas para mejor legibilidad
df = df.rename(columns={
    ' FECHA ': 'Fecha',
    ' INGRESOS_MATRICULAS ': 'Ingresos Matrículas',
    ' INGRESOS_RENOVACIONES ': 'Ingresos Renovaciones',
    ' INGRESOS_ESAL ': 'Ingresos ESAL',
    ' MATRICULA_COMERCIANTE ': 'Matrícula Comerciante',
    ' RENOVADOS_ESAL ': 'Renovados ESAL',
    ' RENOVADOS_COMERCIANTE ': 'Renovados Comerciante',
    ' INGRESOS_RUP ': 'Ingresos RUP',
    ' INGRESOS_DERECHOS_DE_I ': 'Ingresos Derechos de Inscripción',
    ' INGRESOS_RUNEOL ': 'Ingresos RUNEOL',
    ' INGRESOS_FORMU_DE_REGISTRO_M ': 'Ingresos Formulario de Registro M',
    ' INGRESOS_SERVICIOS_ESPECIALES ': 'Ingresos Servicios Especiales'
})

In [6]:
# Función para limpiar y convertir valores monetarios a numéricos
def clean_currency_column(series):
    """
    Convierte una serie con formato de moneda a valores numéricos
    Maneja formatos como: $540,778,126, $ 738,00, etc.
    """
    if series.dtype == 'object':
        # Remover símbolos de moneda, espacios, y comas
        cleaned = series.astype(str)
        cleaned = cleaned.str.replace('$', '', regex=False)
        cleaned = cleaned.str.replace(',', '', regex=False)
        cleaned = cleaned.str.strip()
        
        # Reemplazar valores vacíos o 'nan' con NaN
        cleaned = cleaned.replace(['', 'nan', 'None'], pd.NA)
        
        # Convertir a numérico
        try:
            return pd.to_numeric(cleaned, errors='coerce')
        except:
            return series
    else:
        return series

# Aplicar limpieza a todas las columnas excepto Fecha

for col in df.columns:
    if col != 'Fecha':
        original_dtype = df[col].dtype
        df[col] = clean_currency_column(df[col])
       

In [7]:
# Convertir la columna de fecha a datetime
print("Convirtiendo columna de fecha...")
# Option 1: Convert to Period for proper monthly representation
df['Fecha'] = pd.to_datetime(df['Fecha'], format='%m/%d/%Y', errors='coerce').dt.to_period('M')


Convirtiendo columna de fecha...


In [8]:
df

,Fecha,Ingresos Matrículas,Ingresos Renovaciones,Ingresos ESAL,Matrícula Comerciante,Renovados ESAL,Renovados Comerciante,Ingresos RUP,Ingresos Derechos de Inscripción,Ingresos RUNEOL,Ingresos Formulario de Registro M,Ingresos Servicios Especiales
0,2018-01,143062000,540778126,35868000,NaN,NaN,NaN,NaN,78484800.0,73800.0,23245500.0,NaN
1,2018-02,118430000,945365000,57141000,NaN,NaN,NaN,NaN,73970400.0,82000.0,35105000.0,NaN
2,2018-03,153157000,13908285000,706915000,NaN,NaN,NaN,NaN,106390000.0,3362000.0,167270900.0,NaN
3,2018-04,155416000,2128100792,204861000,NaN,NaN,NaN,NaN,84137300.0,94300.0,48564700.0,NaN
4,2018-05,170378000,1183199000,66850000,NaN,NaN,NaN,NaN,81190600.0,32800.0,30728200.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
88,2025-05,148282000,2281285600,159515900,1162.0,302.0,3508.0,137646000.0,175003700.0,48800.0,41731200.0,10322000.0
89,2025-06,137124600,1280647400,94045500,1181.0,221.0,2229.0,109428000.0,167378000.0,48800.0,30601800.0,10692000.0
90,2025-07,182185000,1271368470,112802400,1526.0,222.0,2369.0,126527000.0,212202100.0,48800.0,34732800.0,14806000.0
91,2025-08,125855900,788034900,55201000,1104.0,151.0,1557.0,91494000.0,170119700.0,6100.0,24000300.0,10758000.0


In [9]:
# Crear diccionario donde cada elemento contiene fecha y una columna
# La llave es el nombre de la columna

diccionario_datos = {}

# Obtener las columnas excepto 'FECHA'
columnas_datos = [col for col in df.columns if col != 'Fecha']

# Crear un elemento del diccionario para cada columna
for columna in columnas_datos:
    diccionario_datos[columna] = df[['Fecha', columna]].copy()



In [10]:
# Eliminar filas con valores nulos en cada DataFrame
for columna in diccionario_datos.keys():
    # Eliminar filas donde cualquier columna tenga valores nulos
    diccionario_datos[columna] = diccionario_datos[columna].dropna()



In [11]:
# EXPORTAR EL DICCIONARIO A CSV (sin manifest, más simple)
archivos_exportados = export_dict_to_csv(
    data_dict=diccionario_datos,
    base_path="data/csv"
)



Exportado: Ingresos Matrículas.csv (93 filas)
Exportado: Ingresos Renovaciones.csv (93 filas)
Exportado: Ingresos ESAL.csv (93 filas)
Exportado: Matrícula Comerciante.csv (81 filas)
Exportado: Renovados ESAL.csv (81 filas)
Exportado: Renovados Comerciante.csv (81 filas)
Exportado: Ingresos RUP.csv (81 filas)
Exportado: Ingresos Derechos de Inscripción.csv (93 filas)
Exportado: Ingresos RUNEOL.csv (91 filas)
Exportado: Ingresos Formulario de Registro M.csv (93 filas)
Exportado: Ingresos Servicios Especiales.csv (81 filas)
